In [ ]:
import pandas as pd
OUTPUT_PERSONS_PATH = "C:/Users/user/PTV_Intern/src/DeepAM/dataset/persons_info.csv"
hh_df = pd.read_csv("C:/Users/user/PTV_Intern/data/raw/NHTS_US_2017/hhpub.csv")
per_df = pd.read_csv("C:/Users/user/PTV_Intern/data/raw/NHTS_US_2017/perpub.csv")
hh_df = hh_df[hh_df['HHSIZE']<=5]
per_df = per_df[per_df['HHSIZE']<=5]
print(len(hh_df), len(per_df))

127848 253644


In [ ]:
# --- Attributes of Interestfor each dataframe ---
nhts_household_columns = [
    'HOUSEID',      # (인덱스용)
    'HHFAMINC',     # 1. Household Income
    'HHSIZE',       # 2. Household Size
    'HHVEHCNT',     # 3. Household Vehicles
    'HOMEOWN',      # 4. Home Ownership Status
    'STDCNT',       # 5. Household Students (SCHTYP로 계산)
    'DRVRCNT',      # 6. Household Licensed Drivers
    'LIF_CYC',      # 7. Household Life Cycle Stage
    'WRKCOUNT',     # 8. Household Employed Members
    'URBRUR',       # 9. Type of Residence (URBAN은 4개 분류, URBRUR은 2개 분류)
    'HBRESDN',      # 10. Housing Density
    'HBPPOPDN',     # 11. Population Density
    'HBHTNRNT',     # 12. Renter-Occupied Housing Ratio
    # '',           # (없음) Housing Status
]
nhts_person_columns = [
    'HOUSEID',      # (인덱스용)
    'PERSONID',     # (인덱스용)
    'DRIVER',       # 1. Driver's License Status
    'EDUC',         # 2. Education Level
    'OCCAT',        # 3. Job Category
    'R_SEX_IMP',    # 4. Gender
    'R_AGE_IMP',    # 5. Age
    'R_RACE',       # 6. Racial/Ethnic Identity
    'PTUSED',       # 7. PTUSED
    'R_RELAT',      # 8. Household Role
    'GT1JBLWK',     # 9. (Family에선 제외) Number of Jobs
    'WORKER',       # 10. Employment Status
    'WRK_HOME',     # 11. (Family에선 제외) Employment Location Type
    # 'GRADE',        # (없음) Current School Grade Level
    # '',             # (없음) Number of Workdays
]
# Other Household Members Info. 블럭 구성시 필요한 9개 feature
family_attributes = [
    'HOUSEID',      # (인덱스용)
    'PERSONID',     # (인덱스용)
    'DRIVER',       # 1. Driver's License Status
    'EDUC',         # 2. Education Level
    'OCCAT',        # 3. Job Category
    'R_SEX_IMP',    # 4. Gender
    'R_AGE_IMP',    # 5. Age
    'R_RACE',       # 6. Racial/Ethnic Identity
    'PTUSED',       # 7. PTUSED
    'R_RELAT',      # 8. Household Role
    'WORKER',       # 9. Employment Status
]

In [ ]:
# Calculate SCHTYP attribute from persons
per_df['IS_STUDENT'] = (per_df['SCHTYP'] == 1).astype(int)
stdcnt_series = per_df.groupby('HOUSEID')['IS_STUDENT'].sum()
hh_df['STDCNT'] = hh_df['HOUSEID'].map(stdcnt_series)

# Drop unusing columns
hh_df = hh_df[nhts_household_columns]
per_df = per_df[nhts_person_columns]

print(len(hh_df.columns), len(per_df.columns))
combined_df = per_df.merge(hh_df, on='HOUSEID', how='left')
len(combined_df.columns)

In [ ]:
# Redefine Indice starting from 0 (conventional procedure to make equal interval between codes)
att_dict = dict()
for col in combined_df.columns:
    if col in ['HOUSEID', 'PERSONID']: continue
    label_dict = dict()
    print(f"{col}: {combined_df[col].unique()}", end='')
    
    # -7이하는 99로 변환, -1은 98로 변환
    combined_df[col] = combined_df[col].apply(lambda x: 99 if x<=-7 else x)
    combined_df[col] = combined_df[col].apply(lambda x: 98 if x==-1 else x)
    
    # 오름차순 정렬 => [유효 값, ..., Appropriate skip,결측치] 순으로 정렬됨
    unique_vals = sorted(combined_df[col].unique())

    # 0부터 시작하는 label로 변환
    for new, old in enumerate(unique_vals):
        label_dict[old] = new
    combined_df[col] = combined_df[col].apply(lambda x: label_dict[x])
    print(f"{combined_df[col].unique()}")
    
    # 0-based label => original label 로 복원하기 위한 dictionary
    att_dict[col] = dict()
    for old in label_dict.keys():
        new = label_dict[old]
        att_dict[col][new] = old

DRIVER: [ 1  2 -1][0 1 2]
EDUC: [ 3  2  5  4  1 -1 -8 -7][2 1 4 3 0 5 6]
OCCAT: [-1  2  4  3 -9  1 97 -8 -7][5 1 3 2 6 0 4]
R_SEX_IMP: [2 1][1 0]
R_AGE_IMP: [67 66 28 55 49 45 68 72 82 78 44 40 50 39 38 30 33 75 61 59 22 54 57 25
 29  6  5 52 56 86 69 87 31 76 51 17 64 37  7 63 65 24 46 14 23 58 60 73
 85 18 19 77 81 47 42 15 13 92 35 62 70 12 36 26 41 16 11 34 48 53 10  8
 74 32 20  9 79 43 27 71 21 83 80 84 88][62 61 23 50 44 40 63 67 77 73 39 35 45 34 33 25 28 70 56 54 17 49 52 20
 24  1  0 47 51 81 64 82 26 71 46 12 59 32  2 58 60 19 41  9 18 53 55 68
 80 13 14 72 76 42 37 10  8 84 30 57 65  7 31 21 36 11  6 29 43 48  5  3
 69 27 15  4 74 38 22 66 16 78 75 79 83]
R_RACE: [ 2  1  6 97  3  4  5 -7 -8][1 0 5 6 2 3 4 7]
PTUSED: [ 0 20  5 22 27  1 21  2 30 10  8  3 18 15 25  6  4 12  7 16 24 17 11 -8
 -7 28 19 13  9 14 23 26 29 -9][ 0 20  5 22 27  1 21  2 30 10  8  3 18 15 25  6  4 12  7 16 24 17 11 31
 28 19 13  9 14 23 26 29]
R_RELAT: [ 1  2  3  7  5  4  6 -9 -7 -8][0 1 2 6 4 3 5 7]
G

In [ ]:
combined_df.to_csv(OUTPUT_PERSONS_PATH)

,HOUSEID,PERSONID,DRIVER,EDUC,OCCAT,R_SEX_IMP,R_AGE_IMP,R_RACE,PTUSED,R_RELAT,...,HHVEHCNT,HOMEOWN,STDCNT,DRVRCNT,LIF_CYC,WRKCOUNT,URBRUR,HBRESDN,HBPPOPDN,HBHTNRNT
0,30000007,1,0,2,5,1,62,1,0,0,...,5,0,0,3,9,1,0,2,3,2
1,30000007,2,0,2,1,0,61,1,0,1,...,5,0,0,3,9,1,0,2,3,2
2,30000007,3,0,1,5,1,23,1,0,2,...,5,0,0,3,9,1,0,2,3,2
3,30000008,1,0,4,3,0,50,0,0,0,...,4,0,0,2,1,2,1,2,2,1
4,30000008,2,0,3,1,1,44,0,0,1,...,4,0,0,2,1,2,1,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253639,40794293,1,0,4,5,1,60,0,0,0,...,3,0,0,2,9,1,1,0,0,2
253640,40794293,2,0,1,6,0,57,0,0,1,...,3,0,0,2,9,1,1,0,0,2
253641,40794294,1,0,3,6,0,36,0,0,0,...,2,0,0,2,1,2,0,6,7,7
253642,40794294,2,0,3,6,1,30,0,0,1,...,2,0,0,2,1,2,0,6,7,7


In [17]:
person_df = pd.read_csv('C:/Users/user/PTV_Intern/src/DeepAM/dataset/person_ids.csv', index_col=0)